In [1]:
from model import Model
from readers import read_ozrock, convert_to_dataset, get_label_list
import torch
from datasets import Dataset, DatasetDict
from transformers import BertTokenizerFast, BertForTokenClassification, TrainingArguments

In [2]:
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Switching to GPU...")
    device = torch.device('cuda')
print("Current device index:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))

CUDA available: True
Switching to GPU...
Current device index: 0
Device name: NVIDIA GeForce RTX 4070 Ti


In [ ]:
datasets, label_list, label2id, id2label = read_ozrock('./OzRock/AutoLabelledSet.txt', './OzRock/EvaluationSet.txt')

In [4]:
# Load tokenizer and model
model_name = 'bert-base-cased'
tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertForTokenClassification.from_pretrained(model_name, num_labels=len(label_list), label2id=label2id, id2label=id2label)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    logging_dir="./logs",
    eval_strategy="steps",
    save_strategy="steps",
    logging_steps=100,
    num_train_epochs=1,
    save_total_limit=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

model = Model(
    device=device,
    model_name=model_name,
    tokenizer=tokenizer,
    model=model,
    label_list=label_list
)

model.set_training_args(training_args)
tokenized_datasets = model.tokenize_datasets(datasets)

Tokenizing datasets...


Map:   0%|          | 0/31942 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
model.train(tokenized_datasets)
model.trainer.save_model(model_name+"-model")
model.tokenizer.save_pretrained(model_name+"-model")

c:\Users\Harry\OneDrive - The University of Western Australia\2025\Honours\test\model.py:80: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Starting training...


Step,Training Loss,Validation Loss,Precision,Recall,F1
100,0.359400,0.271668,0.498814,0.500275,0.499543
200,0.139700,0.250922,0.582309,0.659162,0.618357
300,0.099700,0.241361,0.608593,0.661175,0.633795
400,0.098900,0.250083,0.621526,0.642687,0.631929
500,0.078300,0.250149,0.706315,0.667399,0.686306
600,0.065900,0.242065,0.689016,0.707304,0.698040
700,0.059900,0.270365,0.710486,0.674721,0.692142
800,0.047100,0.254905,0.718338,0.718470,0.718404
900,0.050500,0.250008,0.703407,0.706755,0.705077
1000,0.043400,0.248018,0.702592,0.724327,0.713294


('bert-base-cased-model\\tokenizer_config.json',
 'bert-base-cased-model\\special_tokens_map.json',
 'bert-base-cased-model\\vocab.txt',
 'bert-base-cased-model\\added_tokens.json',
 'bert-base-cased-model\\tokenizer.json')

In [ ]:
model.evaluate(tokenized_datasets["eval"])

{'eval_loss': 0.2694065272808075,
 'eval_precision': 0.7200638637573177,
 'eval_recall': 0.742998352553542,
 'eval_f1': 0.7313513513513512,
 'eval_runtime': 4.3894,
 'eval_samples_per_second': 455.646,
 'eval_steps_per_second': 28.478,
 'epoch': 1.0}